In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("../../data/master/rumdetect2017/rumor_detection_acl2017/twitter16/source_tweets.txt", header=None, delimiter="\t", lineterminator="\n")
data.columns = ["tweet_id", "tweet_text"]
print(data.head())

             tweet_id                                         tweet_text
0  656955120626880512  correct predictions in back to the future ii U...
1  615689290706595840  .@whitehouse in rainbow colors for #scotusmarr...
2  613404935003217920  cops bought the alleged church shooter burger ...
3  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...
4  714598641827246081  an open letter to trump voters from his top st...


In [2]:
labels = pd.read_csv("../../data/master/rumdetect2017/rumor_detection_acl2017/twitter16/label.txt", header=None, delimiter=":", lineterminator="\n")
labels.columns = ["label", "tweet_id"]
print(labels.head())

        label            tweet_id
0       false  656955120626880512
1        true  615689290706595840
2       false  613404935003217920
3  unverified  731166399389962242
4  unverified  714598641827246081


In [3]:
label_dict = {}
labs = {}
for i, l in labels.iterrows():
    label_dict[l["tweet_id"]] = l["label"]
    if l["label"] not in labs:
        labs[l["label"]] = 1
    else:
        labs[l["label"]] += 1
print(labs)
len(label_dict)

{'false': 205, 'true': 207, 'unverified': 201, 'non-rumor': 205}


818

In [4]:
length = data.shape[0]
length

818

In [5]:
length = labels.shape[0]
length

818

In [6]:
data['label'] = data['tweet_id'].map(lambda a: label_dict[a])
data.head()

tweet_id                                         tweet_text  \
0  656955120626880512  correct predictions in back to the future ii U...   
1  615689290706595840  .@whitehouse in rainbow colors for #scotusmarr...   
2  613404935003217920  cops bought the alleged church shooter burger ...   
3  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...   
4  714598641827246081  an open letter to trump voters from his top st...   

        label  
0       false  
1        true  
2       false  
3  unverified  
4  unverified

In [7]:
import random

random.seed(77)

weight = [0, 0, 0, 0, 0, 0, 1, 1, 2, 2]
tvt = []
for i in range(length):
    gacha = random.sample(weight, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("test")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 2)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 2)}")
print(f"Test : {tvt.count('test')} - {round(tvt.count('test')/length, 2)}")

Training : 478 - 0.58
Validation : 175 - 0.21
Test : 165 - 0.2


In [8]:
data['tvt'] = pd.Series(tvt)
data.head()

tweet_id                                         tweet_text  \
0  656955120626880512  correct predictions in back to the future ii U...   
1  615689290706595840  .@whitehouse in rainbow colors for #scotusmarr...   
2  613404935003217920  cops bought the alleged church shooter burger ...   
3  731166399389962242  🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...   
4  714598641827246081  an open letter to trump voters from his top st...   

        label       tvt  
0       false  training  
1        true  training  
2       false  training  
3  unverified  training  
4  unverified  training

In [9]:
data[100:105]

tweet_id                                         tweet_text  \
100  638050997340893184  in celebrity death hoax news, james earl jones...   
101  692890520960397312  the terps do it again! no. 8 maryland knocks o...   
102  544324444773433348  uber says it has hiked prices in sydney to "en...   
103  745236050407194624  the big one: 'large scale motion' detected alo...   
104  673079581520318464  do these look like the faces of a homeless cou...   

          label         tvt  
100       false    training  
101   non-rumor    training  
102        true  validation  
103  unverified    training  
104       false  validation

In [10]:
label_count = data['label'].value_counts()
label_count

true          207
non-rumor     205
false         205
unverified    201
Name: label, dtype: int64

In [11]:
combination = data.apply(lambda row: f"{row['label']}_{row['tvt']}", axis=1).value_counts()
combination

unverified_training      125
false_training           124
true_training            122
non-rumor_training       107
non-rumor_validation      60
false_test                46
true_test                 43
true_validation           42
non-rumor_test            38
unverified_validation     38
unverified_test           38
false_validation          35
dtype: int64

In [12]:
data.to_csv("../../data/processed/twitter16_dataset_with_tvt.csv", index=False)